In [10]:
from common.citylearn import CityLearn
from pathlib import Path
from marlisa import MARLISA as Agent
import matplotlib.pyplot as plt
import pickle
import time

In [2]:
# Load environment
climate_zone = 5
buildings = ["Building_1", "Building_2", "Building_3", 
            "Building_4", "Building_5", "Building_6", 
            "Building_7", "Building_8", "Building_9"]

params = {'data_path':Path("data/Climate_Zone_"+str(climate_zone)), 
        'building_attributes':'building_attributes.json', 
        'weather_file':'weather_data.csv', 
        'solar_profile':'solar_generation_1kW.csv', 
        'carbon_intensity':'carbon_intensity.csv',
        'building_ids':buildings,
        'buildings_states_actions':'common/buildings_state_action_space.json', 
        'simulation_period': (0, 8760*1-1),
        'cost_function': ['ramping','1-load_factor','average_daily_peak','peak_demand','net_electricity_consumption','carbon_emissions'], 
        'central_agent': False,
        'save_memory': False }

env = CityLearn(**params)

# Contains the lower and upper bounds of the states and actions, to be provided to the agent to normalize the variables between 0 and 1.
observations_spaces, actions_spaces = env.get_state_action_spaces()

# Provides information on Building type, Climate Zone, Annual DHW demand, Annual Cooling Demand, Annual Electricity Demand, Solar Capacity, and correllations among buildings
building_info = env.get_building_information()

In [8]:
params_agent = {'building_ids':["Building_"+str(i) for i in [1,2,3,4,5,6,7,8,9]],
                 'buildings_states_actions':'common/buildings_state_action_space.json', 
                 'building_info':building_info,
                 'observation_spaces':observations_spaces, 
                 'action_spaces':actions_spaces, 
                 'hidden_dim':[256,256], 
                 'discount':0.99, 
                 'tau':5e-3, 
                 'lr':3e-4, 
                 'batch_size':256, 
                 'replay_buffer_capacity':1e5, 
                 'regression_buffer_capacity':3e4, 
                 'start_training':600, # Start updating actor-critic networks
                 'exploration_period':7500, # Just taking random actions
                 'start_regression':500, # Start training the regression model
                 'information_sharing':True, # If True -> set the appropriate 'reward_function_ma' in reward_function.py
                 'pca_compression':.95, 
                 'action_scaling_coef':0.5, # Actions are multiplied by this factor to prevent too aggressive actions
                 'reward_scaling':5., # Rewards are normalized and multiplied by this factor
                 'update_per_step':2, # How many times the actor-critic networks are updated every hourly time-step
                 'iterations_as':2,# Iterations of the iterative action selection (see MARLISA paper for more info)
                 'safe_exploration':True} 

# Instantiating the control agent(s)
agent_path = "models/marlisa_model_multiagent"
try:
    agents = pickle.load(open(agent_path, 'rb'))
    print("loaded agent")
except:
    agents = Agent(**params_agent)
    pickle.dump(agents, open(agent_path, 'wb'))
    print("created new agent")

Device:cuda
created new agent


In [11]:
# 1 episode to check time
n_episodes = 1
start = time.time()
for e in range(n_episodes): 
    state = env.reset()
    done = False
    
    j = 0
    is_evaluating = False
    action, coordination_vars = agents.select_action(state, deterministic=is_evaluating)    
    while not done:
        next_state, reward, done, _ = env.step(action)
        action_next, coordination_vars_next = agents.select_action(next_state, deterministic=is_evaluating)
        agents.add_to_buffer(state, action, reward, next_state, done, coordination_vars, coordination_vars_next)
        coordination_vars = coordination_vars_next
        state = next_state
        action = action_next
        
        is_evaluating = (j > 3*8760)
        j += 1
    
    pickle.dump(agents, open(agent_path, 'wb'))
    print("saved agent at episode: ",e)
    print('Loss -',env.cost(), 'Simulation time (min) -',(time.time()-start)/60.0)



saved agent at episode:  0
Loss - {'ramping': 1.0215122, '1-load_factor': 1.0164800226442399, 'average_daily_peak': 1.0067219, 'peak_demand': 1.0, 'net_electricity_consumption': 0.9986488, 'carbon_emissions': 0.9995529, 'total': 1.0071526259142887, 'coordination_score': 1.0111785069046513} Simulation time (min) - 122.51695784330369


In [12]:
print(env.cost())

{'ramping': 1.0215122, '1-load_factor': 1.0164800226442399, 'average_daily_peak': 1.0067219, 'peak_demand': 1.0, 'net_electricity_consumption': 0.9986488, 'carbon_emissions': 0.9995529, 'total': 1.0071526259142887, 'coordination_score': 1.0111785069046513}


### Evaluation code below

In [13]:
from common.citylearn import CityLearn
from pathlib import Path
import pickle

costs = []
for climate_zone in range(1, 5):
    # Load environment
    buildings = ["Building_1", "Building_2", "Building_3", 
                "Building_4", "Building_5", "Building_6", 
                "Building_7", "Building_8", "Building_9"]

    params = {'data_path':Path("data/Climate_Zone_"+str(climate_zone)), 
            'building_attributes':'building_attributes.json', 
            'weather_file':'weather_data.csv', 
            'solar_profile':'solar_generation_1kW.csv', 
            'carbon_intensity':'carbon_intensity.csv',
            'building_ids':buildings,
            'buildings_states_actions':'common/buildings_state_action_space.json', 
            'simulation_period': (0, 8760*1-1),
            'cost_function': ['ramping','1-load_factor','average_daily_peak','peak_demand','net_electricity_consumption','carbon_emissions'], 
            'central_agent': False,
            'save_memory': False }

    env = CityLearn(**params)

    # Contains the lower and upper bounds of the states and actions, to be provided to the agent to normalize the variables between 0 and 1.
    observations_spaces, actions_spaces = env.get_state_action_spaces()

    # Provides information on Building type, Climate Zone, Annual DHW demand, Annual Cooling Demand, Annual Electricity Demand, Solar Capacity, and correllations among buildings
    building_info = env.get_building_information()

    # Instantiating the control agent(s)
    agent_path = "models/marlisa_model_multiagent"
    try:
        agents = pickle.load(open(agent_path, 'rb'))
        print("loaded agent")
    except:
        raise ValueError("no agent found in: " + agent_path)

    state = env.reset()
    done = False

    action, coordination_vars = agents.select_action(state)   
    i = 0
    while not done:
        next_state, reward, done, _ = env.step(action)
        action_next, coordination_vars_next = agents.select_action(next_state)
        coordination_vars = coordination_vars_next
        state = next_state
        action = action_next
    costs.append(env.cost())

    sim_period = (5000, 5500)

avg_costs = {}
for key in costs[0].keys():
    avg_costs[key] = 0.0
    for climate_zone in range(1, 5):
        avg_costs[key] += costs[climate_zone-1][key]
    avg_costs[key] /= 4

print(avg_costs)

loaded agent
loaded agent
loaded agent
loaded agent
{'ramping': 1.2762812972068787, '1-load_factor': 1.1214565018644076, 'average_daily_peak': 1.1987073421478271, 'peak_demand': 1.1093007624149323, 'net_electricity_consumption': 1.0117691457271576, 'carbon_emissions': 1.0171207189559937, 'total': 1.1224392947195327, 'coordination_score': 1.1764364759085115}


In [18]:
# Plotting summer operation in the last year
interval = range(8760*3 + 24*30*6, 8760*3 + 24*30*6 + 24*10)
plt.figure(figsize=(16,5))
plt.plot(env.net_electric_consumption_no_pv_no_storage[interval])
plt.plot(env.net_electric_consumption_no_storage[interval])
plt.plot(env.net_electric_consumption[interval], '--')
plt.xlabel('time (hours)')
plt.ylabel('kW')
plt.legend(['Electricity demand without storage or generation (kW)', 
            'Electricity demand with PV generation and without storage(kW)', 
            'Electricity demand with PV generation and using RBC for storage(kW)'])

IndexError: index 30600 is out of bounds for axis 0 with size 8759

<Figure size 1152x360 with 0 Axes>

In [17]:
len(env.net_electric_consumption_no_pv_no_storage)

8759

In [19]:

# Plotting 5 days of winter operation of the last year
plt.figure(figsize=(16,5))
interval = range(8760*3, 8760*3 + 24*5)
plt.plot(env.net_electric_consumption_no_pv_no_storage[interval])
plt.plot(env.net_electric_consumption_no_storage[interval])
plt.plot(env.net_electric_consumption[interval], '--')
plt.xlabel('time (hours)')
plt.ylabel('kW')
plt.legend(['Electricity demand without storage or generation (kW)', 
            'Electricity demand with PV generation and without storage(kW)', 
            'Electricity demand with PV generation and using RBC for storage(kW)'])

IndexError: index 26280 is out of bounds for axis 0 with size 8759

<Figure size 1152x360 with 0 Axes>